In [1]:
import urllib.request, requests, pickle, json, time, pymongo
from collections import defaultdict


# ↓--- 고정 ---↓

naver_client_id_list = ['6x79fVFmyAsJwoczKRKv', 'iQ2_kgauWnDwovI2V_A2', 'PAjCUqGHlQfq9FV7VPHN', 'pc2a87TGwtMzR9sFtiW4'] # "EZKJjFYCeCFKfj3qk2OM" 
naver_client_secret_list = ['FvpgDL1RKw', '7zNrQQ09HJ', 'ADmfp7rlz2', 'wP5FkLr1Ua'] # "dLMxOkCV6M"
naver_client_id = ''
naver_client_secret = ''


kakao_API_url = "https://dapi.kakao.com//v3/search/book"
Kakao_MY_REST_API_list = ['f31f6598676120e724444fa392d6e5ac', '91ca26dcf2e331983be854a11384dbc5', '15433ce5f89fe1d58fba3bcd9e81f4fb', '9cc5c3b3e586a451faf880c470315296'] # 'a9f59d8981601a41b5fb31311b82b8a4'
Kakao_MY_REST_API = ''

tokens = ''

# ↑--- 고정 ---↑


pickle_data_path = '/workspace/data/image0_sort_popular_overall_asin.pickle'


In [2]:
with open(pickle_data_path, "rb") as f:
    asin_number = pickle.load(f)

len(asin_number)

418865

In [3]:
file = open("asin_link.txt", 'w')
idx = 0
minimum_length = 1

naver_client_id = naver_client_id_list[idx]
naver_client_secret = naver_client_secret_list[idx]
Kakao_MY_REST_API = Kakao_MY_REST_API_list[idx]
break_point = 288001

for i in range(break_point, len(asin_number)):
    time.sleep(0.1)
    if (i != 0) and (i % 24000) == 0:
        idx += 1
        
        naver_client_id = naver_client_id_list[idx]
        naver_client_secret = naver_client_secret_list[idx]
        Kakao_MY_REST_API = Kakao_MY_REST_API_list[idx]

    try:
        number = asin_number[i]
        
        input_text = number

        url_list = []

        # Naver API search

        naver_input_text_n = urllib.parse.quote(input_text)

        naver_url = "https://openapi.naver.com/v1/search/book?query=" + naver_input_text_n +"&display=3&sort=sim"

        Send_request = urllib.request.Request(naver_url)
        Send_request.add_header("X-Naver-Client-Id", naver_client_id)
        Send_request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(Send_request)

        success = response.getcode()

        if (success == 200):
            Response = response.read()
            tokens = json.loads(Response.decode('utf-8'))

            for token in tokens['items']:
                try:
                    url = token['image']
                    if len(url) >= minimum_length:
                        url_list.append(url)
                except:
                    continue

            if len(url_list) >= minimum_length:
                file.write(str(number) + '\t' + str(url_list) + '\n')
        
        else:
            # Kakao API search

            queryString = {'query' : input_text}
            header = {'Authorization': 'KakaoAK ' + Kakao_MY_REST_API }

            response = requests.get(kakao_API_url, headers=header, params=queryString)

            tokens = response.json()

            url = tokens["documents"][0]["thumbnail"]
            
            if len(url) >= minimum_length:
                url_list.append(url)

            file.write(str(number) + '\t' + str(url_list) + '\n')

    except:
        break_point = i
        print(break_point)
        break

file.close()

IndexError: list index out of range

In [4]:
# 데이터 추가

client = pymongo.MongoClient("mongodb://118.67.143.144:30001/")

db = client["amazon"]
collection = db["url"]

# DB에 어떤 collection이 있는 지 확인
# print(db.list_collection_names())

file = open("asin_link.txt", 'r') # 데이터 형식 : asin_number\t[url_1, url_2, ...]\n

files = file.readlines()

for line in files:
    line = line.rstrip()
    line = line.split('\t') 
    asin = line[0]
    url_dump = line[1][1:len(line[1])-1].replace(' ', '')
    url_list = url_dump.split(',')
    url_list_processed = [i[1:len(i)-1] for i in url_list]

    x = collection.insert_one({"asin": asin, "image_url": url_list_processed})

file.close()